In [1]:
from google.colab import drive
drive.mount('/content/drive')

import sys
path = '/content/drive/MyDrive/Colab Notebooks/NeurIPS2024'
sys.path.append(path)

import numpy as np
import pandas as pd
import pickle

from realdata import *

import os

import matplotlib.pyplot as plt


Mounted at /content/drive


In [2]:
algo = 'XGB'

test_next = False


results_all = []
for idx in range(10):
    le = idx * 10 + 1
    ri = le + 9
    filename = path + '/results/{}_{}_{}_{}.pkl'.format(algo, le, ri, test_next)
    with open(filename, 'rb') as r:
        tmp = pickle.load(r)
        results_all += tmp


D = 10
M = int(np.ceil(826 / D))
list_t = np.array(range(1, M)) * D

prop = [0.3, 0.1]
window_sizes_train = [1, 16, 256, 1024]
window_sizes_infer = [1, 16, 256, 1024]
weights = [0.99, 0.9, 0.5, 0.25]
alpha = 0.1
delta = 0.1



num_train = len(window_sizes_train)
num_infer = len(window_sizes_infer)
num_weights = len(weights)
num_t = len(list_t)


S = len(results_all)
idx_start = 0
print('Algorithm = {}'.format(algo))
print()
print()

for (idx_model, window_size) in enumerate(window_sizes_train):
    print('Results for training window {}: '.format(window_sizes_train[idx_model]))

    MAE = dict()
    MAE['ARW'] = 0
    MAE['fixed'] = np.zeros(num_infer)
    MAE['weighted'] = np.zeros(num_weights)

    MAEsd = dict()
    MAEsd['ARW'] = 0
    MAEsd['fixed'] = np.zeros(num_infer)
    MAEsd['weighted'] = np.zeros(num_weights)

    AC = dict()
    AC['ARW'] = 0
    AC['fixed'] = np.zeros(num_infer)
    AC['weighted'] = np.zeros(num_weights)

    AW = dict()
    AW['baseline'] = 0
    AW['ARW'] = 0
    AW['fixed'] = np.zeros(num_infer)
    AW['weighted'] = np.zeros(num_weights)

    for s in range(S):
        MAE_s, AC_s, AW_s = summarize(results_all[s], alpha, idx_start)
        for key in MAE.keys():
            MAE[key] += MAE_s[idx_model][key] / S
            MAEsd[key] += MAE_s[idx_model][key] ** 2 / S
            AC[key] += AC_s[idx_model][key] / S
        for key in AW.keys():
            AW[key] += AW_s[idx_model][key] / S

    for key in MAE.keys():
        MAEsd[key] = np.sqrt(MAEsd[key] - (MAE[key] ** 2))

    print('MAE: ', MAE)
    print('AC: ', AC)
    print('AW: ', AW)

    print()

    print('MAEsd: ', MAEsd)

    print()
    print()


Algorithm = XGB


Results for training window 1: 
MAE:  {'ARW': 0.03433366286677259, 'fixed': array([0.0629387 , 0.03012674, 0.04128511, 0.0446994 ]), 'weighted': array([0.0446994 , 0.04471774, 0.04520451, 0.0458529 ])}
AC:  {'ARW': 0.925338173325766, 'fixed': array([0.87428754, 0.91041343, 0.93424874, 0.93955386]), 'weighted': array([0.93955386, 0.93958249, 0.94030505, 0.94115931])}
AW:  {'baseline': 2.175496702154525, 'ARW': 1.5821652768225736, 'fixed': array([1.37626776, 1.49431174, 1.6528683 , 1.69600891]), 'weighted': array([1.69600891, 1.69629303, 1.70236928, 1.71027917])}

MAEsd:  {'ARW': 0.002199861972349872, 'fixed': array([0.00554598, 0.00225276, 0.00231469, 0.00215277]), 'weighted': array([0.00215277, 0.0021498 , 0.00216472, 0.00217301])}


Results for training window 16: 
MAE:  {'ARW': 0.03135510595322951, 'fixed': array([0.06340474, 0.02966096, 0.05247855, 0.06192294]), 'weighted': array([0.06192294, 0.06194185, 0.06264765, 0.06358287])}
AC:  {'ARW': 0.9197163359678706, 'f